In [7]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
import time

In [8]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
all_data = pd.read_csv("Nomao.data", header = None)
all_data.replace('?', np.NaN, inplace=True)

all_data = all_data.drop(0, axis=1)

data_train, data_test, labels_train, labels_test = train_test_split(all_data.iloc[:,:118], all_data.iloc[:,[118]], test_size=0.3, random_state = 42, stratify=all_data.iloc[:,-1])

imp = SimpleImputer(missing_values=np.NaN, strategy="most_frequent")
cat_data_train = list ()
num_data_train = list ()
cat_data_test = list ()
num_data_test = list ()
cat_cols = [7, 8, 15, 16, 23, 24, 31, 32, 39, 40, 47, 48, 55, 56, 63, 64, 71, 72, 79, 80, 87, 88, 92, 96, 100, 104, 108, 112, 116] 
for col in cat_cols :
  cat_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  cat_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))
cols = all_data.columns
num_cols = list(set (cols) - set (cat_cols) - {119})
imp = SimpleImputer(missing_values=np.NaN, strategy="mean")
for col in num_cols :
  num_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  num_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))

j1 , j2 = 0, 0

idata_train = pd.DataFrame()
idata_test = pd.DataFrame()
for col in range(1,119) :
  if j2 < 29 and cat_cols[j2] == col :
    idata_train = pd.concat([idata_train,cat_data_train[j2]],join="outer",axis=1)
    idata_test = pd.concat([idata_test, cat_data_test[j2]],join="outer",axis=1)
    j2 = j2 + 1
  else :
    idata_train = pd.concat([idata_train,num_data_train[j1]],join="outer",axis=1)
    idata_test = pd.concat([idata_test,num_data_test[j1]],join="outer",axis=1)    
    j1 = j1 + 1

temp = pd.concat([idata_train,idata_test], join="outer", axis=0)
dummy_data = pd.get_dummies(temp, [i-1 for i in cat_cols])
np_dummy_data = dummy_data.apply(pd.to_numeric).values

np_data_train = np_dummy_data[:24125,:] #οι γραμμές δεν αλλάζουν θέση οπότε αφού σειριακά
np_data_test = np_dummy_data[24125:,:]  #ενώσαμε τα train, test σειριακά θα τα χωρίσουμε

mapping = {-1 : 0, +1 : 1}

labels_train[119] = labels_train[119].map(mapping)
labels_test[119] = labels_test[119].map(mapping)

np_labels_train = labels_train[119].values.flatten()
np_labels_test = labels_test[119].values.flatten()

X_train = np_data_train
y_train = np_labels_train
X_test = np_data_test
y_test = np_labels_test

In [9]:
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)

f1_micro_score_mlp = cross_val_score(mlp, X_train, y_train, cv = 5, scoring ='f1_micro')
f1_macro_score_mlp = cross_val_score(mlp, X_train, y_train, cv = 5, scoring = "f1_macro")


print("The Multilayer perceptron Classifier has f1_micro score on training set : %f \n" % (f1_micro_score_mlp.mean()))
print("The Multilayer perceptron Classifier has f1_macro score on training set : %f \n" % (f1_macro_score_mlp.mean()))
print("The Multilayer perceptron Classifier has scores on test set : \n" + classification_report(y_test,pred_mlp))

The Multilayer perceptron Classifier has f1_micro score on training set : 0.956228 

The Multilayer perceptron Classifier has f1_macro score on training set : 0.950592 

The Multilayer perceptron Classifier has scores on test set : 
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      2953
           1       0.97      0.98      0.97      7387

    accuracy                           0.96     10340
   macro avg       0.96      0.95      0.95     10340
weighted avg       0.96      0.96      0.96     10340



In [10]:
#pipe με default τιμές (αρχικοποίηση) και επιλεγμένο τον StandardScaler

selector = VarianceThreshold()
scaler = StandardScaler()
ros = RandomOverSampler()
pca = PCA()
mlp = MLPClassifier()
pipe_mlp = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('mlp', mlp)])
pipe_mlp.fit(X_train, y_train)
pred_pipe_mlp = pipe_mlp.predict(X_test)
f1_micro_score_pipe_mlp = cross_val_score(pipe_mlp, X_train, y_train, cv = 5, scoring = 'f1_micro')
f1_macro_score_pipe_mlp = cross_val_score(pipe_mlp, X_train, y_train, cv = 5, scoring = 'f1_macro')

print("The Multilayer perceptron Classifier with default preprocessing has f1_micro score on training set : %f \n" % (f1_micro_score_pipe_mlp.mean()))
print("The Multilayer perceptron Classifier with default preprocessing has f1_macro score on training set : %f \n" % (f1_macro_score_pipe_mlp.mean()))
print("The Multilayer perceptron Classifier with default preprocessing parameters has scores on test set: \n" + classification_report(y_test,pred_pipe_mlp))

The Multilayer perceptron Classifier with default preprocessing has f1_micro score on training set : 0.959710 

The Multilayer perceptron Classifier with default preprocessing has f1_macro score on training set : 0.949668 

The Multilayer perceptron Classifier with default preprocessing parameters has scores on test set: 
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      2953
           1       0.98      0.97      0.97      7387

    accuracy                           0.96     10340
   macro avg       0.95      0.96      0.95     10340
weighted avg       0.96      0.96      0.96     10340



In [11]:
#pipe με default τιμές (αρχικοποίηση) και επιλεγμένο τον MinMaxScaler

selector = VarianceThreshold()
mscaler = StandardScaler()
ros = RandomOverSampler()
pca = PCA()
mlp = MLPClassifier()
mpipe_mlp = Pipeline(steps=[('scaler', mscaler), ('selector', selector), ('sampler', ros), ('pca', pca), ('mlp', mlp)])
mpipe_mlp.fit(X_train, y_train)
mpred_pipe_mlp = mpipe_mlp.predict(X_test)
mf1_micro_score_pipe_mlp = cross_val_score(mpipe_mlp, X_train, y_train, cv = 5, scoring = 'f1_micro')
mf1_macro_score_pipe_mlp = cross_val_score(mpipe_mlp, X_train, y_train, cv = 5, scoring = 'f1_macro')

print("The Multilayer perceptron Classifier with default preprocessing (using MinMax Scaler) has f1_micro score on training set : %f \n" % (mf1_micro_score_pipe_mlp.mean()))
print("The Multilayer perceptron Classifier with default preprocessing (using MinMax Scaler) has f1_macro score on training set : %f \n" % (mf1_macro_score_pipe_mlp.mean()))
print("The Multilayer perceptron Classifier with default preprocessing parameters (using MinMax Scaler) has scores on test set: \n" + classification_report(y_test,mpred_pipe_mlp))

The Multilayer perceptron Classifier with default preprocessing (using MinMax Scaler) has f1_micro score on training set : 0.960041 

The Multilayer perceptron Classifier with default preprocessing (using MinMax Scaler) has f1_macro score on training set : 0.950711 

The Multilayer perceptron Classifier with default preprocessing parameters (using MinMax Scaler) has scores on test set: 
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      2953
           1       0.98      0.97      0.97      7387

    accuracy                           0.96     10340
   macro avg       0.95      0.95      0.95     10340
weighted avg       0.96      0.96      0.96     10340



In [33]:
vthreshold = [0, 0.03, 0.05, 1e-1]
n_components = [1,15,20, 25]

pipe_mlp = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('mlp', mlp)], memory = 'tmp')

In [34]:
#joined_train = np.concatenate((X_train, np.array([y_train]).T), axis = 1)
#X_sample, _, y_sample, _ = train_test_split(joined_train[:,:174], joined_train[:,174],test_size = 0.25, stratify=joined_train[:,-1])

In [35]:
#Grid Search Cross Validation με επιλεγμένο τον StandardScaler
vthreshold = [0.05, 0.01, 0.015, 0.02, 0.03]
n_components = [30, 35, 38, 40]
hidden_layer_sizes = [(40,)]
activation = ['logistic']
solver = ['adam']
max_iter = [50]
learning_rate = ['adaptive']
alpha = [0.0001]

estimator = GridSearchCV(pipe_mlp, dict(selector__threshold=vthreshold, pca__n_components=n_components, mlp__hidden_layer_sizes = hidden_layer_sizes ,
mlp__activation = activation, mlp__solver = solver, mlp__max_iter =  max_iter,
mlp__learning_rate = learning_rate,mlp__alpha = alpha), cv=5, scoring='f1_micro',n_jobs=-1)
start_time = time.time()
estimator.fit(X_train, y_train) #estimator.fit(X_train, y_train)
estimator_pred = estimator.predict(X_test)
estimator_time = time.time() - start_time

print("The Multilayer perceptron Classifier has f1_micro score on training set : %f \n" % (estimator.best_score_))
print("The Multilayer perceptron Classifier optimized by f1_micro : \n" + classification_report(y_test,estimator_pred))
print("with best parameters")
print(estimator.best_params_)

#pipe_mlp = estimator.best_estimator_
#pred_pipe_mlp = estimator_pred
#time_pipe_mlp = estimator_time

print("\n")

estimator = GridSearchCV(pipe_mlp, dict(selector__threshold=vthreshold, pca__n_components=n_components, mlp__hidden_layer_sizes = hidden_layer_sizes ,
mlp__activation = activation, mlp__solver = solver, mlp__max_iter =  max_iter,
mlp__learning_rate = learning_rate,mlp__alpha = alpha), cv=5, scoring='f1_macro',n_jobs=-1)
start_time = time.time()
estimator.fit(X_train, y_train) #estimator.fit(X_train, y_train)
estimator_pred = estimator.predict(X_test)
estimator_time = time.time() - start_time

print("The GaussianNB Classifier has f1_macro score on training set : %f \n" % (estimator.best_score_))
print("The GaussianNB Classifier optimized by f1_macro : \n" + classification_report(y_test,estimator_pred))
print("with best parameters")
print(estimator.best_params_)

print("\n")

The Multilayer perceptron Classifier has f1_micro score on training set : 0.953078 

The Multilayer perceptron Classifier optimized by f1_micro : 
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      2953
           1       0.98      0.95      0.97      7387

    accuracy                           0.95     10340
   macro avg       0.94      0.95      0.94     10340
weighted avg       0.96      0.95      0.95     10340

with best parameters
{'mlp__activation': 'logistic', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (40,), 'mlp__learning_rate': 'adaptive', 'mlp__max_iter': 50, 'mlp__solver': 'adam', 'pca__n_components': 38, 'selector__threshold': 0.03}


The GaussianNB Classifier has f1_macro score on training set : 0.943109 

The GaussianNB Classifier optimized by f1_macro : 
              precision    recall  f1-score   support

           0       0.89      0.96      0.92      2953
           1       0.98      0.95      0.97      

In [36]:
#Grid Search Cross Validation με επιλεγμένο τον MinMaxScaler

vthreshold = [0.065, 0.07, 0.075, 0.078]
n_components = [35, 38, 40, 45]
hidden_layer_sizes = [(40,), (100,)]
activation = ['identity', 'logistic']
solver = ['adam']
max_iter = [50]
learning_rate = ['constant']
alpha = [0.0001]

mestimator = GridSearchCV(mpipe_mlp, dict(selector__threshold=vthreshold, pca__n_components=n_components, mlp__hidden_layer_sizes = hidden_layer_sizes ,
mlp__activation = activation, mlp__solver = solver, mlp__max_iter =  max_iter,
mlp__learning_rate = learning_rate,mlp__alpha = alpha), cv=5, scoring='f1_micro',n_jobs=-1)
start_time = time.time()
mestimator.fit(X_train, y_train) #mestimator.fit(X_train, y_train)
mestimator_pred = mestimator.predict(X_test)
mestimator_time = time.time() - start_time

print("The Multilayer perceptron Classifier has f1_micro score on training set : %f \n" % (mestimator.best_score_))
print("The Multilayer perceptron Classifier optimized by f1_micro : \n" + classification_report(y_test,mestimator_pred))
print("with best parameters")
print(mestimator.best_params_)

#pipe_mlp = mestimator.best_estimator_
#pred_pipe_mlp = mestimator_pred
#time_pipe_mlp = mestimator_time

print("\n")

mestimator = GridSearchCV(mpipe_mlp, dict(selector__threshold=vthreshold, pca__n_components=n_components, mlp__hidden_layer_sizes = hidden_layer_sizes ,
mlp__activation = activation, mlp__solver = solver, mlp__max_iter =  max_iter,
mlp__learning_rate = learning_rate,mlp__alpha = alpha), cv=5, scoring='f1_macro',n_jobs=-1)
start_time = time.time()
mestimator.fit(X_train, y_train) #mestimator.fit(X_train, y_train)
mestimator_pred = mestimator.predict(X_test)
mestimator_time = time.time() - start_time

print("The GaussianNB Classifier has f1_macro score on training set : %f \n" % (mestimator.best_score_))
print("The GaussianNB Classifier optimized by f1_macro : \n" + classification_report(y_test,mestimator_pred))
print("with best parameters")
print(mestimator.best_params_)

print("\n")


The Multilayer perceptron Classifier has f1_micro score on training set : 0.954280 

The Multilayer perceptron Classifier optimized by f1_micro : 
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      2953
           1       0.98      0.96      0.97      7387

    accuracy                           0.96     10340
   macro avg       0.94      0.95      0.95     10340
weighted avg       0.96      0.96      0.96     10340

with best parameters
{'mlp__activation': 'logistic', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (100,), 'mlp__learning_rate': 'constant', 'mlp__max_iter': 50, 'mlp__solver': 'adam', 'pca__n_components': 45, 'selector__threshold': 0.075}


The GaussianNB Classifier has f1_macro score on training set : 0.945357 

The GaussianNB Classifier optimized by f1_macro : 
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      2953
           1       0.98      0.96      0.97    

In [38]:
#Grid Search Cross Validation χωρίς Scaler

vthreshold = [0.01, 0.015, 0.03, 0.04]
n_components = [30, 35, 38, 40]
hidden_layer_sizes = [(40,), (100,)]
activation = ['identity']
solver = ['adam']
max_iter = [50]
learning_rate = ['constant']
alpha = [0.0001]

pipe_mlp = Pipeline(steps=[('selector', selector), ('sampler', ros), ('pca', pca), ('mlp', mlp)], memory = 'tmp')
estimator = GridSearchCV(pipe_mlp, dict(selector__threshold=vthreshold, pca__n_components=n_components, mlp__hidden_layer_sizes = hidden_layer_sizes ,
mlp__activation = activation, mlp__solver = solver, mlp__max_iter =  max_iter,
mlp__learning_rate = learning_rate,mlp__alpha = alpha), cv=5, scoring='f1_micro',n_jobs=-1)
start_time = time.time()
estimator.fit(X_train, y_train) #estimator.fit(X_train, y_train)
estimator_pred = estimator.predict(X_test)
estimator_time = time.time() - start_time

print("The Multilayer perceptron Classifier (with no Scaler applied) has f1_micro score on training set : %f \n" % (estimator.best_score_))
print("The Multilayer perceptron Classifier (with no Scaler applied) optimized by f1_micro : \n" + classification_report(y_test,estimator_pred))
print("with best parameters")
print(estimator.best_params_)

#pipe_mlp = estimator.best_estimator_
#pred_pipe_mlp = estimator_pred
#time_pipe_mlp = estimator_time

print("\n")

pipe_mlp = Pipeline(steps=[('selector', selector), ('sampler', ros), ('pca', pca), ('mlp', mlp)], memory = 'tmp')
estimator = GridSearchCV(pipe_mlp, dict(selector__threshold=vthreshold, pca__n_components=n_components, mlp__hidden_layer_sizes = hidden_layer_sizes ,
mlp__activation = activation, mlp__solver = solver, mlp__max_iter =  max_iter,
mlp__learning_rate = learning_rate,mlp__alpha = alpha), cv=5, scoring='f1_macro',n_jobs=-1)
start_time = time.time()
estimator.fit(X_train, y_train) #estimator.fit(X_train, y_train)
estimator_pred = estimator.predict(X_test)
estimator_time = time.time() - start_time

print("The Multilayer perceptron Classifier (with no Scaler applied) has f1_macro score on training set : %f \n" % (estimator.best_score_))
print("The Multilayer perceptron Classifier (with no Scaler applied) optimized by f1_macro : \n" + classification_report(y_test,estimator_pred))
print("with best parameters")
print(estimator.best_params_)

#pipe_mlp = estimator.best_estimator_
#pred_pipe_mlp = estimator_pred
#time_pipe_mlp = estimator_time

print("\n")


The Multilayer perceptron Classifier (with no Scaler applied) has f1_micro score on training set : 0.936000 

The Multilayer perceptron Classifier (with no Scaler applied) optimized by f1_micro : 
              precision    recall  f1-score   support

           0       0.84      0.94      0.89      2953
           1       0.97      0.93      0.95      7387

    accuracy                           0.93     10340
   macro avg       0.91      0.93      0.92     10340
weighted avg       0.94      0.93      0.93     10340

with best parameters
{'mlp__activation': 'identity', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (40,), 'mlp__learning_rate': 'constant', 'mlp__max_iter': 50, 'mlp__solver': 'adam', 'pca__n_components': 40, 'selector__threshold': 0.015}


The Multilayer perceptron Classifier (with no Scaler applied) has f1_macro score on training set : 0.924790 

The Multilayer perceptron Classifier (with no Scaler applied) optimized by f1_macro : 
              precision    recall  